# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from geopy import distance

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'geopy'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = pd.read_csv("output.csv")
cities_pd.head()

,City,ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,atuona,4020109,-9.8000,-139.0333,79.29,77,22,18.81,PF,Wed Jun 23 18:37:31 2021
1,plettenberg bay,964712,-34.0527,23.3716,55.27,83,100,4.59,ZA,Wed Jun 23 18:37:31 2021
2,bonavista,5905393,48.6499,-53.1147,64.40,99,99,15.73,CA,Wed Jun 23 18:37:32 2021
3,namatanai,2090021,-3.6667,152.4333,84.25,70,100,3.65,PG,Wed Jun 23 18:37:32 2021
4,broken hill,2173911,-31.9500,141.4333,55.11,58,2,16.11,AU,Wed Jun 23 18:37:32 2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)

In [4]:
locations = cities_pd[["Lat", "Long"]].astype(float)
city_humidity = cities_pd["Humidity"].astype(float) 

In [5]:
center_coord = (40.52,34.34)

fig = gmaps.figure(center=center_coord,zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)



fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
cities_df = pd.DataFrame(cities_pd)
cities_df

,City,ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,atuona,4020109,-9.8000,-139.0333,79.29,77,22,18.81,PF,Wed Jun 23 18:37:31 2021
1,plettenberg bay,964712,-34.0527,23.3716,55.27,83,100,4.59,ZA,Wed Jun 23 18:37:31 2021
2,bonavista,5905393,48.6499,-53.1147,64.40,99,99,15.73,CA,Wed Jun 23 18:37:32 2021
3,namatanai,2090021,-3.6667,152.4333,84.25,70,100,3.65,PG,Wed Jun 23 18:37:32 2021
4,broken hill,2173911,-31.9500,141.4333,55.11,58,2,16.11,AU,Wed Jun 23 18:37:32 2021
...,...,...,...,...,...,...,...,...,...,...
554,pemangkat,1631637,1.1667,108.9667,78.26,85,98,6.89,ID,Wed Jun 23 18:40:54 2021
555,bentiu,377472,9.2333,29.8333,72.09,81,38,5.30,SS,Wed Jun 23 18:40:55 2021
556,chanika,160592,-5.4167,38.0167,59.13,98,67,2.95,TZ,Wed Jun 23 18:40:55 2021
557,chuguyevka,2025286,44.1653,133.8633,79.27,42,11,3.87,RU,Wed Jun 23 18:40:56 2021


In [7]:
# Ideal weather conditions: 75-85F, Wind Speed 5-25mph, humidity 10-60%

cities_ideal = cities_df[(cities_df['Max Temp'] < 82) & 
            (cities_df['Wind'] >= 5) & (cities_df['Wind'] <= 25) & 
            (cities_df['Humidity'] >= 15) & (cities_df['Humidity'] <= 55)]

cities_ideal_df = pd.DataFrame(cities_ideal)
cities_ideal_df.dropna(inplace=True)
print(len(cities_ideal_df))

47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
cities_ideal_df=cities_ideal_df.drop(columns=["Date"])
cities_ideal_df["Hotel Name"] = ""

In [9]:
print(cities_ideal_df)

                    City       ID      Lat      Long  Max Temp  Humidity  \
40           yellowknife  6185377  62.4560 -114.3525     59.56        33   
68             kununurra  2068110 -15.7667  128.7333     75.18        50   
70             saskylakh  2017155  71.9167  114.0833     55.49        39   
72             shelburne  6145890  44.0787  -80.2041     66.69        50   
89          port hedland  2063042 -20.3167  118.5667     59.05        51   
112                dukat  2125906  62.5500  155.5500     57.87        37   
123              birjand   140463  32.8663   59.2211     59.47        20   
158             cherskiy  2126199  68.7500  161.3000     66.36        27   
159                korla  1529376  41.7597   86.1469     75.49        30   
183            tilichiki  2120591  60.4667  166.1000     65.73        45   
185                macon  4243719  39.8667  -88.9668     78.55        44   
217            flin flon  5954718  54.7682 -101.8650     64.94        36   
235         

In [35]:
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=formatted_address,name,geometry&locationbias=circle:5000@"

search_lat = 71.9167
search_lng = 114.0833

response = requests.get(f"{base_url}{search_lat},{search_lng}&key={g_key}")
data = response.json()
print(data)

{'candidates': [{'formatted_address': '5399 Adobe Falls Rd, San Diego, CA 92120, United States', 'geometry': {'location': {'lat': 32.7812233, 'lng': -117.0824267}, 'viewport': {'northeast': {'lat': 32.78265067989273, 'lng': -117.0813331701073}, 'southwest': {'lat': 32.77995102010728, 'lng': -117.0840328298927}}}, 'name': 'Rodeway Inn San Diego Near Sdsu'}], 'status': 'OK'}


In [27]:
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=formatted_address,name,geometry&locationbias=circle:5000@"

# hotel_df=pd.DataFrame(columns=["Name","Address","Latitude","Longitude"])
hotel_df=pd.DataFrame(columns=["Address"])
results_total = []
# citydata_df=pd.DataFrame(columns=["ID","Lat","Long","Max Temp","Humidity","Cloudiness","Wind","Country","Date"],index=city)

# use iterrows to iterate through pandas dataframe
for index, row in cities_ideal_df.iterrows():
    search_city = row["City"]
    search_lat = row["Lat"]
    search_lng = row["Long"]

    # # assemble url and make API request
    response = requests.get(f"{base_url}{search_lat},{search_lng}&key={g_key}").json()
    
    # # extract results
    # results = response.loads(response)
    results=(json.dumps(response, indent=4, sort_keys=True))
    # resultsdict = json.loads(results)

    results_total.append(results)

    # for objects in results:
        # hotel_name = objects[0][2]
        # hotel_add = objects[1]
        # hotel_lat = objects[0][1][0][0]
        # hotel_lng = objects[0][1][0][1]
    # for objects in results:
    #     hotel_name = objects['candidates']['name']
    #     hotel_add = objects['candidates']['formatted_address']
    #     hotel_lat = objects['candidates']['geometry']['location']['lat']
    #     hotel_lng = objects['candidates']['geometry']['location']['lng']
    # results_list.append(results)

    # hotel_name = results
    # hotels_list.append(hotel_name)
    # hotel_add = results[0]['candidates']['formatted_address']
    # latlng = results[0]['geometry']['location']
    # hotel_df.loc=[hotel_add]

    # hotel_df.loc=[hotel_name,hotel_add,hotel_lat,hotel_lng]


In [28]:
print(results_total)


['{\n    "candidates": [\n        {\n            "formatted_address": "4825 49th Ave, Yellowknife, NT X1A 2R3, Canada",\n            "geometry": {\n                "location": {\n                    "lat": 62.4575798,\n                    "lng": -114.372667\n                },\n                "viewport": {\n                    "northeast": {\n                        "lat": 62.45848992989271,\n                        "lng": -114.3708717701073\n                    },\n                    "southwest": {\n                        "lat": 62.45579027010727,\n                        "lng": -114.3735714298927\n                    }\n                }\n            },\n            "name": "The Explorer Hotel"\n        }\n    ],\n    "status": "OK"\n}', '{\n    "candidates": [\n        {\n            "formatted_address": "20 Victoria Hwy, Kununurra WA 6743, Australia",\n            "geometry": {\n                "location": {\n                    "lat": -15.782683,\n                    "lng": 128

In [26]:
results_df = pd.DataFrame(results_total[1])
print(results_df)

                                          candidates status
0  {'formatted_address': '20 Victoria Hwy, Kununu...     OK


print(results_list)

In [15]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map


# Display figure
